### Imports

In [1]:
import pandas as pd
import numpy as np
import openpyxl

### Load Data

In [2]:
df = pd.read_excel(r"data_masterstudie_2022-07-13_12-12.xlsx")

In [3]:
df

,CASE,SERIAL,REF,QUESTNNR,MODE,STARTED,AE01_01,AE01_02,AE02_01,AE03_01,...,MAILSENT,LASTDATA,FINISHED,Q_VIEWER,LASTPAGE,MAXPAGE,MISSING,MISSREL,TIME_RSI,DEG_TIME
0,Interview-Nummer (fortlaufend),Seriennummer (sofern verwendet),Referenz (sofern im Link angegeben),"Fragebogen, der im Interview verwendet wurde",Interview-Modus,Zeitpunkt zu dem das Interview begonnen hat (E...,Rsikoverhalten: Investition in Alternative A,Rsikoverhalten: Investition in Alternative B,Quelle: Erbe,ErbeRiskant: Alternative 2,...,Versandzeitpunkt der Einladungsmail (nur für n...,Zeitpunkt als der Datensatz das letzte mal geä...,Wurde die Befragung abgeschlossen (letzte Seit...,Hat der Teilnehmer den Fragebogen nur angesehe...,"Seite, die der Teilnehmer zuletzt bearbeitet hat","Letzte Seite, die im Fragebogen bearbeitet wurde",Anteil fehlender Antworten in Prozent,Anteil fehlender Antworten (gewichtet nach Rel...,Maluspunkte für schnelles Ausfüllen,Maluspunkte für schnelles Ausfüllen
1,250,NaN,NaN,base,interview,2022-07-06 09:52:30,NaN,NaN,NaN,NaN,...,NaN,2022-07-06 09:56:06,1,0,7,7,3,0,1.88,47
2,253,NaN,NaN,base,interview,2022-07-06 13:27:25,NaN,NaN,NaN,NaN,...,NaN,2022-07-06 13:35:37,1,0,7,7,6,3,0.87,1
3,254,NaN,NaN,base,interview,2022-07-06 13:30:33,NaN,NaN,NaN,NaN,...,NaN,2022-07-06 13:35:32,1,0,7,7,3,0,1.53,27
4,255,NaN,NaN,base,interview,2022-07-06 14:19:42,NaN,NaN,NaN,NaN,...,NaN,2022-07-06 14:33:27,1,0,7,7,71,73,1.33,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,488,NaN,NaN,base,interview,2022-07-13 11:33:42,NaN,NaN,NaN,NaN,...,NaN,2022-07-13 11:42:15,1,0,7,7,3,0,0.82,4
141,489,NaN,NaN,base,interview,2022-07-13 11:57:28,NaN,NaN,NaN,NaN,...,NaN,2022-07-13 12:06:07,1,0,7,7,3,0,0.92,7
142,490,NaN,NaN,base,interview,2022-07-13 12:01:09,NaN,NaN,NaN,NaN,...,NaN,2022-07-13 12:06:11,1,0,7,7,3,0,1.47,26
143,491,NaN,NaN,base,interview,2022-07-13 12:09:06,NaN,NaN,NaN,NaN,...,NaN,2022-07-13 12:10:42,0,0,4,4,0,0,1.68,34


### Filter for relevant columns

In [4]:
df = df[["FS01_CP","FS01","SD03","SD05","SD11","SD11_10","SD14","SD14_08","SD16","MZ03_01","MZ03_02","MZ03_03","MZ03_04","MZ03_05","MZ03_06","SC01","SC02","VR01_01",
"AN01_01","AN01_02","AN02_01","AN02_02","AN03_01","AN03_02","AN04_01","AN04_02","RI01","RI02","RI03","RI04","RI05","RI06","RI07","RI08","RI09","RI10","RI11","RI12","RI13","TIME_SUM","FINISHED"
]]

In [5]:
df.columns

Index(['FS01_CP', 'FS01', 'SD03', 'SD05', 'SD11', 'SD11_10', 'SD14', 'SD14_08',
       'SD16', 'MZ03_01', 'MZ03_02', 'MZ03_03', 'MZ03_04', 'MZ03_05',
       'MZ03_06', 'SC01', 'SC02', 'VR01_01', 'AN01_01', 'AN01_02', 'AN02_01',
       'AN02_02', 'AN03_01', 'AN03_02', 'AN04_01', 'AN04_02', 'RI01', 'RI02',
       'RI03', 'RI04', 'RI05', 'RI06', 'RI07', 'RI08', 'RI09', 'RI10', 'RI11',
       'RI12', 'RI13', 'TIME_SUM', 'FINISHED'],
      dtype='object')

### Drop first row 

In [6]:
df = df.iloc[1: , :]

### Subtract 1 from some columns because they start at 1 not at 0

In [7]:
df[['AN01_01','AN01_02','AN02_01','AN02_02', 'AN03_01', 'AN03_02', 'AN04_01', 'AN04_02']] = df[['AN01_01','AN01_02','AN02_01','AN02_02', 'AN03_01', 'AN03_02', 'AN04_01', 'AN04_02']] -1

In [8]:
df[['AN01_01','AN01_02','AN02_01','AN02_02', 'AN03_01', 'AN03_02', 'AN04_01', 'AN04_02']]

,AN01_01,AN01_02,AN02_01,AN02_02,AN03_01,AN03_02,AN04_01,AN04_02
1,NaN,NaN,8,92,52,48,82,18
2,NaN,NaN,20,80,20,80,20,80
3,28,72,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,-10,-10,-10,-10,-10,-10
5,50,50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
140,20,80,NaN,NaN,NaN,NaN,NaN,NaN
141,NaN,NaN,0,100,29,71,100,0
142,20,80,NaN,NaN,NaN,NaN,NaN,NaN
143,NaN,NaN,25,75,0,100,100,0


### Exclude data with short answer time <3 minutes and data where no consent/the last page was not reached

In [9]:
df_short_answer = df[df["TIME_SUM"] >= 180 ]

In [10]:
df_finished = df_short_answer[df_short_answer["FINISHED"] == 1] 

In [11]:
df_consent = df_finished[df_finished["SC01"] == 2 ]

In [12]:
df_overall = df_consent[df_consent["SC02"] == 1]

In [13]:
df_final = df_overall.loc[(df_overall["RI01"] != -9)  & (df_overall["RI02"] != -9)]

In [14]:
df_final = df_final.loc[(df_final["AN01_01"] != -10)  & (df_final["AN01_02"] != -10) &(df_final["AN02_01"] != -10) &(df_final["AN02_02"] != -10) &(df_final["AN03_01"] != -10) &(df_final["AN03_02"] != -10) &(df_final["AN04_01"] != -10) &(df_final["AN04_02"] != -10)]

### Create a Column with the total amount invested in the risky alternative

In [15]:

df_final['SAVE_TOTAL']= df_final['AN02_02'].where(df_final['FS01'] ==1, 0) * 0.2 + df_final['AN03_02'].where(df_final['FS01'] ==1, 0) * 0.7 + df_final['AN04_02'].where(df_final['FS01'] ==1, 0) * 0.1 + df_final['AN02_02'].where(df_final['FS01'] ==2, 0) * 0.2 + df_final['AN03_02'].where(df_final['FS01'] ==2, 0) * 0.5 + df_final['AN04_02'].where(df_final['FS01'] ==2, 0) * 0.3  + df_final['AN01_02'].where(df_final['FS01'] ==3, 0)

df_final['RISK_TOTAL']= df_final['AN02_01'].where(df_final['FS01'] ==1, 0) * 0.2 + df_final['AN03_01'].where(df_final['FS01'] ==1, 0) * 0.7 + df_final['AN04_01'].where(df_final['FS01'] ==1, 0) * 0.1 + df_final['AN02_01'].where(df_final['FS01'] ==2, 0) * 0.2 + df_final['AN03_01'].where(df_final['FS01'] ==2, 0) * 0.5 + df_final['AN04_01'].where(df_final['FS01'] ==2, 0) * 0.3 + df_final['AN01_01'].where(df_final['FS01'] ==3, 0)

In [16]:
df_final[['RISK_TOTAL','SAVE_TOTAL']]

,RISK_TOTAL,SAVE_TOTAL
1,46.2,53.8
2,20.0,80.0
3,28.0,72.0
5,50.0,50.0
6,50.0,50.0
...,...,...
138,29.0,71.0
139,20.6,79.4
140,20.0,80.0
141,30.3,69.7


### Save to excel file

In [17]:
df_final.to_excel("data_masterstudie_filtered.xlsx")